<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">

<h1><font color="#00586D" size=4>Trabajo Fin de Master (TFM)</font></h1>
<br>
<h2><font color="#00586D" size=6>WiDS Datathon 2024 - Challenge 2</font></h2>
<h3><font color="#00586D" size=5>Modelos de regresión para estimación del periodo de diagnóstico metastático</font></h3>
<h1><font color="#00586D" size=4>Alumna: Luna Jiménez Fernández</font></h1>
<br>


<div align="right">
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>
</div>

<br>
---

INTRO A LA LIBRETA

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Importing main libraries
import numpy as np
import pandas as pd

---

<a id="indice"></a>
<h2><font color="#00586D" size=5>Índice</font></h2>

* [1. Explicación del conjunto de datos](#section1)
* [2. Análisis de datos exploratorio (EDA)]
* [3. Pre-procesamiento de los datos y construcción de pipelines]
* [4. Modelos de regresión]
* [5. Experimentación]
* [6. Análisis]
* [7. Conclusiones]
---

TITULOS ASI:

<a id="section1"></a>
## <font color="#00586D"> 1. Explicación del conjunto de datos </font>
<br>

In [5]:
# Loading the train dataset - used for data exploration
df_metastasis = pd.read_csv(
    "data/train.csv",
    index_col="patient_id"
# 
)

display(df_metastasis.head(10))

,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18,metastatic_diagnosis_period
patient_id,,,,,,,,,,,,,,,,,,,,,
268700,NaN,COMMERCIAL,AR,724,South,West South Central,39,F,NaN,C50912,...,52.55,74.77,79.96,81.69,78.30,74.56,59.98,42.98,41.18,191
484983,White,NaN,IL,629,Midwest,East North Central,55,F,35.36,C50412,...,49.30,72.87,77.40,77.43,75.83,72.64,58.36,39.68,39.71,33
277055,NaN,COMMERCIAL,CA,925,West,Pacific,59,F,NaN,1749,...,68.50,70.31,78.61,87.24,85.52,80.75,70.81,62.67,55.58,157
320055,Hispanic,MEDICAID,CA,900,West,Pacific,59,F,NaN,C50911,...,63.34,63.10,67.45,75.86,75.24,71.10,68.95,65.46,59.46,146
190386,NaN,COMMERCIAL,CA,934,West,Pacific,71,F,NaN,1748,...,59.45,60.24,64.77,69.81,70.13,68.10,65.38,60.72,54.08,286
559027,NaN,COMMERCIAL,IN,461,Midwest,East North Central,63,F,NaN,1749,...,45.86,71.10,74.27,74.89,74.57,70.70,55.43,37.13,35.43,73
293747,White,MEDICARE ADVANTAGE,OH,448,Midwest,East North Central,57,F,33.10,C50412,...,42.62,65.91,71.26,74.03,73.94,69.12,53.50,36.43,34.10,59
517596,White,COMMERCIAL,DE,198,South,South Atlantic,56,F,31.05,C50411,...,48.41,65.17,70.63,75.82,76.17,70.00,56.65,40.90,37.68,316
533188,NaN,COMMERCIAL,LA,706,South,West South Central,65,F,NaN,C50212,...,63.74,77.51,81.80,83.07,82.46,80.32,71.56,56.24,53.39,86
